In [11]:
#Deep learning Sentiment Analysis
#Written by Noah Walsh, Matt Manoly, and James Kaufman
#9/27/18
#We perform a basic polarity analysis, where we import a dataset of Amazon reviews of various products
# and train the convolutional neural network to identify good or bad reviews, and test on a validation set.
#The reviews are imported, the words replaced with corresponding frequency values up to 3000, the less common
# values and stopwords set to zeros in the examples. The training/test datasets were split 50/50. We achieved
# a best accuracy of 87.64%, and a common top accuracy of around 85%.

# imports
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import regularizers
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras import optimizers
import nltk
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

line = []
metaX = []
xdata = []
count = 0
useful = False
rating = False
flag = True
yinput2 = []
#path to the Amazon reviews
urlstring = "/home/matt/Desktop/sorted_data_acl/"
for review in ["books/negative.review","books/positive.review","dvd/positive.review","dvd/negative.review","electronics/positive.review","electronics/negative.review","kitchen_&_housewares/positive.review","kitchen_&_housewares/negative.review"]:
    with open(str(urlstring+review)) as data: #Reads through every product review
        for token in data:
            #Title and review text are only parts that matter
            if(token == "</title>\n"): #end tag, stop reading
                useful = False
            if(token == "</review_text>\n"): #end tag
                useful = False
                sent_str = ""
                for i in metaX:
                    sent_str += str(i) + " "
                sent_str = sent_str[:-1]

                xdata.append([sent_str])
                metaX = []
                count +=1
            if(useful):
                line.append(token)
                metaX.append(token)
            if(token == "<review_text>\n" or token == "<title>\n"): #start tag, begin reading
                useful = True
    
    #Set up target array with 0s corresponding to negative reviews, and 1s corresponding to positive reviews
    if review == "books/negative.review" or review == "dvd/negative.review" or review == "electronics/negative.review" or review == "kitchen_&_housewares/negative.review":
        yinput2 = np.concatenate([yinput2,np.full(count,0)])
    elif review == "books/positive.review" or review == "dvd/positive.review" or review == "electronics/positive.review" or review == "kitchen_&_housewares/positive.review":
        yinput2 = np.concatenate([yinput2,np.full(count,1)])
    count = 0
# Read through the unlabeled data, and set it to 1 (positive) if the rating is > 3, and set it to 0 (negative)
# if the rating is < 3, and ignores review if rating = 3.
for review in ["dvd/unlabeled.review","electronics/unlabeled.review","kitchen_&_housewares/unlabeled.review"]:
    with open(str(urlstring+review)) as data: #Reads through every product review
        for token in data:
            #Title and review text are only parts that matter
            if(token == "</title>\n"): #end tag, stop reading
                useful = False
            if(token == "</rating>\n"):
                rating = False
            if(token == "</review_text>\n"): #end tag
                useful = False
                sent_str = ""
                for i in metaX:
                    sent_str += str(i) + " "
                sent_str = sent_str[:-1]

                xdata.append([sent_str])
                metaX = []
                count +=1
            if(useful and flag):
                line.append(token)
                metaX.append(token)
            if (rating):
                if (float(token) > 3):
                    flag = True
                    yinput2 = np.append(yinput2,np.full(1,1))
                elif (float(token) < 3):
                    flag = True
                    yinput2 = np.append(yinput2,np.full(1,0))
                elif (float(token) == 3):
                    flag = False
            if(token == "<review_text>\n" or token == "<title>\n"): #start tag, begin reading
                useful = True
            if (token == "<rating>\n"):
                rating = True


[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
posi

positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
positive
positive
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
p

negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
p

positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
negative
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
p

positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
p

...

positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
negative
positive
positive
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
n

negative
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
negative
negative
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
p

positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
p

positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
positive
positive
positive
negative
positive
positive
negative
negative
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
negative
negative
negative
negative
positive
negative
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
positive
negative
positive
negative
positive
positive
positive
negative
positive
negative
positive
positive
n

positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
negative
negative
negative
positive
negative
negative
negative
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
negative
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
negative
positive
positive
negative
positive
negative
negative
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
positive
negative
positive
positive
positive
positive
negative
positive
positive
positive
positive
positive
positive
positive
positive
n

In [12]:
lines = []

#remove all special characters from the text, and set everything to lowercase
for i in line:
    for char in [".","?","'","#","\n",",","!","@","$","%","^","&","*","(",")","+","-","_",";",":","<",">",'"',"{","}","\\","/","~","`"]:
        if char in i:
            i = i.replace(char,"")
    i = i.lower()
    lines.extend(nltk.word_tokenize(i)) #tokenize strings into lists of words

#counts the occurences of each word, freq is a dictionary
freq = Counter(lines)

In [13]:
xinput = []

#remove special characters and make lowercase for xinput
for i in xdata:
    for char in [".","?","'","#","\n",",","!","@","$","%","^","&","*","(",")","+","-","_",";",":","<",">",'"',"{","}","\\","/","~","`"]:
        if char in i[0]:
            i[0] = i[0].replace(char,"")
    i[0] = i[0].lower()
    xinput.append(nltk.word_tokenize(i[0])) #tokenize strings into lists of words

In [14]:
freqList = []
top = freq.most_common(3000) #only use 3000 most used words
for i in range(len(top)):
    freqList.append(top[i][0])

In [15]:
stop_words = set(stopwords.words("english"))

#Assigns numeric value based on frequency count to every word in xinput. Stopwords are set to 0
for i in range(len(xinput)):
    for j in range(len(xinput[i])):
        if xinput[i][j] in freqList:
            if xinput[i][j] in stop_words:
                xinput[i][j] = 0
            else:
                temp = freqList.index(xinput[i][j])
                xinput[i][j]=temp
        else:
            xinput[i][j]=0 

In [16]:
max_length = 550 #max length of review
xinput2 = sequence.pad_sequences(xinput, maxlen=max_length) #pad x inputs so they're all set length

In [19]:
from sklearn.utils import shuffle #import unison shuffle function

#splits training and testing data exactly 50/50 between all categories, all unlabeled is grouped together and split
myXtrain,myYtrain = xinput2[:500], yinput2[:500]
myXtest,myYtest = xinput2[500:1000], yinput2[500:1000]

myXtrain = np.concatenate((myXtrain, xinput2[1000:1500]))
myXtrain = np.concatenate((myXtrain, xinput2[2000:2500]))
myXtrain = np.concatenate((myXtrain, xinput2[3000:3500]))
myXtrain = np.concatenate((myXtrain, xinput2[4000:4500]))
myXtrain = np.concatenate((myXtrain, xinput2[5000:5500]))
myXtrain = np.concatenate((myXtrain, xinput2[6000:6500]))
myXtrain = np.concatenate((myXtrain, xinput2[7000:7500]))
myXtrain = np.concatenate((myXtrain, xinput2[8000:40339]))

myYtrain = np.concatenate((myYtrain, yinput2[1000:1500]))
myYtrain = np.concatenate((myYtrain, yinput2[2000:2500]))
myYtrain = np.concatenate((myYtrain, yinput2[3000:3500]))
myYtrain = np.concatenate((myYtrain, yinput2[4000:4500]))
myYtrain = np.concatenate((myYtrain, yinput2[5000:5500]))
myYtrain = np.concatenate((myYtrain, yinput2[6000:6500]))
myYtrain = np.concatenate((myYtrain, yinput2[7000:7500]))
myYtrain = np.concatenate((myYtrain, yinput2[8000:40339]))

myXtest = np.concatenate((myXtest, xinput2[1500:2000]))
myXtest = np.concatenate((myXtest, xinput2[2500:3000]))
myXtest = np.concatenate((myXtest, xinput2[3500:4000]))
myXtest = np.concatenate((myXtest, xinput2[4500:5000]))
myXtest = np.concatenate((myXtest, xinput2[5500:6000]))
myXtest = np.concatenate((myXtest, xinput2[6500:7000]))
myXtest = np.concatenate((myXtest, xinput2[7500:8000]))
myXtest = np.concatenate((myXtest, xinput2[40339:]))

myYtest = np.concatenate((myYtest, yinput2[1500:2000]))
myYtest = np.concatenate((myYtest, yinput2[2500:3000]))
myYtest = np.concatenate((myYtest, yinput2[3500:4000]))
myYtest = np.concatenate((myYtest, yinput2[4500:5000]))
myYtest = np.concatenate((myYtest, yinput2[5500:6000]))
myYtest = np.concatenate((myYtest, yinput2[6500:7000]))
myYtest = np.concatenate((myYtest, yinput2[7500:8000]))
myYtest = np.concatenate((myYtest, yinput2[40339:]))

myXtrain, myYtrain = shuffle(myXtrain, myYtrain) #shuffle the data
myXtest, myYtest = shuffle(myXtest, myYtest) #shuffle the data

36339
36339
36340
36340


In [27]:
mymodel = Sequential()
top_words = 3000

# Add our layers.
# Word embedding input layer.
mymodel.add(Embedding(top_words, 32, input_length=max_length))

# 1D convolutional layer.
mymodel.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

# Max Pooling layer.
mymodel.add(MaxPooling1D(pool_size=2))

# Flattening our vector.
mymodel.add(Flatten())

# Rectifier activation function.
mymodel.add(Dense(250, activation='relu', kernel_constraint=maxnorm(3), kernel_regularizer=regularizers.l2(0), 
                activity_regularizer=regularizers.l1(0)))

mymodel.add(Dropout(0)) # dropout layer
# Rectifier activation function.
mymodel.add(Dense(250, activation='relu', kernel_constraint=maxnorm(3), kernel_regularizer=regularizers.l2(0), #Add l2 and l1 regularization
                activity_regularizer=regularizers.l1(0)))
mymodel.add(Dropout(0)) # dropout layer
# Sigmoid activation function.
mymodel.add(Dense(1, activation='sigmoid'))

# Compile, and then fit the model to the data.
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True) #Specify params for adam optimizer
mymodel.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
mymodel.fit(myXtrain, myYtrain, validation_data=(myXtest, myYtest), epochs=30, batch_size=300, verbose=2) #Batch size 50 to 100

# Final evaluation of the model
scores = mymodel.evaluate(myXtest, myYtest, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 36339 samples, validate on 36340 samples
Epoch 1/30
 - 42s - loss: 0.3901 - acc: 0.8387 - val_loss: 0.3943 - val_acc: 0.8239
Epoch 2/30
 - 40s - loss: 0.2170 - acc: 0.9137 - val_loss: 0.3377 - val_acc: 0.8548
Epoch 3/30
 - 41s - loss: 0.1351 - acc: 0.9505 - val_loss: 0.3315 - val_acc: 0.8646
Epoch 4/30
 - 41s - loss: 0.0767 - acc: 0.9742 - val_loss: 0.5802 - val_acc: 0.8007
Epoch 5/30
 - 41s - loss: 0.0345 - acc: 0.9902 - val_loss: 0.5406 - val_acc: 0.8534
Epoch 6/30
 - 41s - loss: 0.0167 - acc: 0.9960 - val_loss: 0.6424 - val_acc: 0.8514
Epoch 7/30
 - 41s - loss: 0.0063 - acc: 0.9990 - val_loss: 0.7761 - val_acc: 0.8485
Epoch 8/30
 - 41s - loss: 0.0026 - acc: 0.9997 - val_loss: 0.8840 - val_acc: 0.8440
Epoch 9/30
 - 41s - loss: 0.0011 - acc: 0.9999 - val_loss: 0.9923 - val_acc: 0.8369
Epoch 10/30
 - 41s - loss: 7.3742e-04 - acc: 1.0000 - val_loss: 1.0241 - val_acc: 0.8404
Epoch 11/30
 - 40s - loss: 6.4071e-04 - acc: 1.0000 - val_loss: 1.0637 - val_acc: 0.8397
Epoch 12/30
 - 4

KeyboardInterrupt: 